# 工具类

In [117]:
import os
import glob
import datetime
import pymysql
import pandas as pd
base_dir= r"C:\Users\Administrator\Desktop\\公交\\project_passenger_analysis"
os.chdir(base_dir)
data_dir=["data_handle/data/sta_ic_detail","data_handle/data/sta_ic_detail_1"]
#mysql_ip='127.0.0.1';mysql_usr='root':mysql_pwd='123456';mysql_db='raysdata'   #本地数据库连接（做功能测试）
#mysql_ip='10.10.168.104';mysql_usr='traffic';mysql_pwd='123456';mysql_db='traffic_analyze'  #sta_ic_detail表操作不动
mysql_ip='10.10.100.197';mysql_usr='root';mysql_pwd='123456';mysql_db='traffic_analyze'

def get_time():  
    time_now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f : ')
    return time_now 

def get_file_paths(data_dir):
    file_path=[]
    for i in  data_dir:
        file_path=file_path+glob.glob("%s/*"%i) 
    return file_path

def connect():
    connection = pymysql.connect(host=mysql_ip, port=3306, user=mysql_usr, password=mysql_pwd, db=mysql_db, charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
    cur = connection.cursor()
    return connection,cur

def connect1():
    connection = pymysql.connect(host=mysql_ip, port=3306, user=mysql_usr, password=mysql_pwd, db=mysql_db, charset='utf8mb4',)
    cur = connection.cursor()
    return connection,cur

def executeScriptsFromFile(filename, cursor):
    fd = open(filename, 'r', encoding='utf-8')
    sqlFile = fd.read()
    fd.close()
    sqlCommands = sqlFile.split(';')
    for command in sqlCommands:
        try:
            print(get_time(),"SQL:",command)
            cursor.execute(command)
            print(get_time(),"Complete")
        except Exception as msg:
            print(msg)
    print('sql执行完成')

# 一 导入测试数据

In [2]:
import pymysql
import pandas as pd

batch = 10000

connection,cur = connect()

sql_insert = 'insert into sta_ic_detail (FLOW_NUM,CARD_ID,CARD_TYPE,TRADE_TYPE,ORG_ID,TRADE_TIME,TRADE_STATION_NAME,\
                    MARK_TIME,MARK_STATION_NAME,LINE_ID,BUS_ID,MARK_LINE_ID,MARK_BUS_ID,DOWN_TIME,DOWN_STATION_NAME,\
                    TRADE_DATE,DRIVER_FACE_ID,POS_ID,TRIP_ID,DIRECTION,TRANSFER_FLAG,TRANSFER_ID)\
                    values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'

#filepath = ["./data/sta_ic_detail_jn_30000_name.txt"]
filepath =get_file_paths(data_dir)

def insertfirst_txt(filepath):
    try:          
        for file in filepath:            
            f = open(file, 'r', encoding='UTF-8')
            batchdata = []
            index = 0
            totalnum = 0
            line = f.readline() 
            colnums = len(line.strip('\n').split(',')) - 1     #txt多了一列就-1
            while line:
                index = index + 1
                list = line.strip('\n').split(',')[0:colnums]   #txt过滤最后一列重复的日期
                #list = line.strip('\n').split(',')
                batchdata.append(list)
                if (index == batch):
                    totalnum = totalnum + index
                    cur.executemany(sql_insert, batchdata)
                    print("文件%s第%s条" %(file,totalnum))
                    batchdata = []
                    index = 0
                line = f.readline()
            totalnum = totalnum + index
            cur.executemany(sql_insert, batchdata)
            print(totalnum)
            f.close()
    finally:
        cur.close()
        connection.close()
        
        
def insertfirst_csv(filepath):
    try:          
        for file in filepath:            
            f = open(file, 'r', encoding='UTF-8')
            batchdata = []
            index = 0
            totalnum = 0
            line = f.readline() 
            while line:
                index = index + 1
                list = line.strip('\n').split(',')
                batchdata.append(list)
                if (index == batch):
                    totalnum = totalnum + index
                    cur.executemany(sql_insert, batchdata)
                    print("文件%s第%s条" %(file,totalnum))
                    batchdata = []
                    index = 0
                line = f.readline()
            totalnum = totalnum + index
            cur.executemany(sql_insert, batchdata)
            print(totalnum)
            f.close()
    finally:
        cur.close()
        connection.close()
        
        
def noinsertfirst():
    try:
        batchdata = []
        index = 0
        totalnum = 0
        head = f.readline()
        colnums = len(head.strip('\n').split(',')) - 1
        for line in f:
            index = index + 1
            list = line.strip('\n').split(',')[0:colnums]
            batchdata.append(list)
            if (index == batch):
                totalnum = totalnum + index
                cur.executemany(sql_insert, batchdata)
                print(totalnum)
                batchdata = []
                index = 0
        totalnum = totalnum + index
        cur.executemany(sql_insert, batchdata)
        print(totalnum)
    finally:
        f.close()
        cur.close()
        connection.close()

if __name__=="__main__":
    print(get_time(),"################Starting Loading####################")
    # noinsertfirst()
    if  "txt" in filepath[0]:
        insertfirst_txt(filepath)
    elif  "csv" in filepath[0]:
        insertfirst_csv(filepath)

    print(get_time(),"################Complete Loading####################")


2019-09-16 09:29:12.753282 :  ################Starting Loading####################
文件data/sta_ic_detail\20190415.csv第10000条
文件data/sta_ic_detail\20190415.csv第20000条
文件data/sta_ic_detail\20190415.csv第30000条
文件data/sta_ic_detail\20190415.csv第40000条
文件data/sta_ic_detail\20190415.csv第50000条
文件data/sta_ic_detail\20190415.csv第60000条
文件data/sta_ic_detail\20190415.csv第70000条
文件data/sta_ic_detail\20190415.csv第80000条
文件data/sta_ic_detail\20190415.csv第90000条
文件data/sta_ic_detail\20190415.csv第100000条
文件data/sta_ic_detail\20190415.csv第110000条
文件data/sta_ic_detail\20190415.csv第120000条
文件data/sta_ic_detail\20190415.csv第130000条
文件data/sta_ic_detail\20190415.csv第140000条
文件data/sta_ic_detail\20190415.csv第150000条
文件data/sta_ic_detail\20190415.csv第160000条
文件data/sta_ic_detail\20190415.csv第170000条
文件data/sta_ic_detail\20190415.csv第180000条
文件data/sta_ic_detail\20190415.csv第190000条
196268
文件data/sta_ic_detail\20190416.csv第10000条
文件data/sta_ic_detail\20190416.csv第20000条
文件data/sta_ic_detail\20190416.csv第30000

文件data/sta_ic_detail_1\20190429.csv第50000条
文件data/sta_ic_detail_1\20190429.csv第60000条
文件data/sta_ic_detail_1\20190429.csv第70000条
文件data/sta_ic_detail_1\20190429.csv第80000条
文件data/sta_ic_detail_1\20190429.csv第90000条
文件data/sta_ic_detail_1\20190429.csv第100000条
文件data/sta_ic_detail_1\20190429.csv第110000条
文件data/sta_ic_detail_1\20190429.csv第120000条
文件data/sta_ic_detail_1\20190429.csv第130000条
文件data/sta_ic_detail_1\20190429.csv第140000条
文件data/sta_ic_detail_1\20190429.csv第150000条
文件data/sta_ic_detail_1\20190429.csv第160000条
文件data/sta_ic_detail_1\20190429.csv第170000条
文件data/sta_ic_detail_1\20190429.csv第180000条
文件data/sta_ic_detail_1\20190429.csv第190000条
文件data/sta_ic_detail_1\20190429.csv第200000条
201181
文件data/sta_ic_detail_1\20190430.csv第10000条
文件data/sta_ic_detail_1\20190430.csv第20000条
文件data/sta_ic_detail_1\20190430.csv第30000条
文件data/sta_ic_detail_1\20190430.csv第40000条
文件data/sta_ic_detail_1\20190430.csv第50000条
文件data/sta_ic_detail_1\20190430.csv第60000条
文件data/sta_ic_detail_1\20190430.csv第

# 二 出行链处理(用sql脚本替代)

In [3]:
connection,cur = connect()
executeScriptsFromFile("b_ic2TripChain.sql",cur)
cur.close()
connection.close()

2019-09-16 09:39:43.576866 :  SQL: -- 去掉相同卡和相同下车时间的重复多余数据（保留row_num最小的那条）
INSERT INTO sta_ic_detail_tmp
SELECT (@rownumber :=@rownumber + 1) AS rn,
	s.*
	FROM sta_ic_detail s,
	(SELECT @rownumber := 0) r	
2019-09-16 09:39:57.314389 :  Complete
2019-09-16 09:39:57.315384 :  SQL: 
	
TRUNCATE TABLE sta_ic_detail
2019-09-16 09:39:57.450301 :  Complete
2019-09-16 09:39:57.451298 :  SQL: 

INSERT INTO sta_ic_detail
SELECT 
	FLOW_NUM,CARD_ID,CARD_TYPE ,TRADE_TYPE ,ORG_ID ,TRADE_TIME,TRADE_STATION_NAME,MARK_TIME ,
	MARK_STATION_NAME,LINE_ID,BUS_ID,MARK_LINE_ID,MARK_BUS_ID,DOWN_TIME ,DOWN_STATION_NAME,
	TRADE_DATE,DRIVER_FACE_ID ,POS_ID,TRIP_ID ,DIRECTION ,TRANSFER_FLAG ,TRANSFER_ID
FROM sta_ic_detail_tmp p WHERE p.ROW_NUM IN (
 SELECT MIN(t.ROW_NUM) rn FROM  sta_ic_detail_tmp t GROUP BY t.CARD_ID,t.TRADE_TIME
)
2019-09-16 09:46:23.646800 :  Complete
2019-09-16 09:46:23.646800 :  SQL: 


-- 根据ic数据得到出行链表   sta_ic_detail ---> trip_chain_ex 
-- 性能对比 基于12.5w的ic数据 python270min<->sql7s 
INSERT INTO t

In [ ]:
print(get_time(),"################Starting Handle Chain####################")
import pymysql
import datetime

batch = 1000

connection,cur = connect()

def get_group_condition():
    try:
        sql_condition = "select CARD_ID,TRADE_DATE,TRANSFER_ID,count(*) as TRANSFER_NUM from sta_ic_detail group by CARD_ID,TRADE_DATE,TRANSFER_ID"
        cur.execute(sql_condition)
        results = cur.fetchall()
        print("总共数量＝" + str(len(results)))
        return results
    except Exception as e:
        print(e.message)
        raise e

def createTripChainExData(condition_array):
    try:
        rownum = 0
        totalnum = 0
        trip_chain_ex_array = []
        sql_insert = 'insert into trip_chain_ex (CARD_ID,TRADE_DATE,TRANSFER_ID,TRANSFER_NUM,ON_STATION_NAME,ON_TIME,ON_TIME_BUCKET,DOWN_STATION_NAME,DOWN_TIME,DOWN_TIME_BUCKET,LINE,TRANSFER_STATION) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        for condition in condition_array:
            trip_chain_ex = []
            trip_chain_ex.append(condition["CARD_ID"])
            trip_chain_ex.append(condition["TRADE_DATE"])
            trip_chain_ex.append(condition["TRANSFER_ID"])
            trip_chain_ex.append(condition["TRANSFER_NUM"])
            sql_condition = "select MARK_STATION_NAME,MARK_TIME,DOWN_STATION_NAME,DOWN_TIME,MARK_LINE_ID from sta_ic_detail \
            where CARD_ID = '%s' and TRADE_DATE = '%s' and TRANSFER_ID = '%s' order by MARK_TIME" % (trip_chain_ex[0],trip_chain_ex[1],trip_chain_ex[2])
            cur.execute(sql_condition)
            results = cur.fetchall()
            num = len(results)
            index = 0
            on_time = ""
            on_station_name = ""
            down_time = ""
            down_station_name = ""
            line = ""
            transfer =""
            for row in results:
                if index == 0:
                    on_station_name = row["MARK_STATION_NAME"]
                    on_time = row["MARK_TIME"]
                if index == num-1:
                    down_station_name = row["DOWN_STATION_NAME"]
                    down_time = row["DOWN_TIME"]
                line = line + str(row["MARK_LINE_ID"]) + "-"
                transfer = transfer + row["MARK_STATION_NAME"] + "*" + row["DOWN_STATION_NAME"] + "-"
                index = index + 1
            line = line[0:len(line)-1]
            transfer = transfer[0:len(transfer)-1]
            if len(transfer.split("-")) == 0:
                transfer = ""
            else:
                temp = transfer.split("*")[1:len(transfer.split("*"))-1]
                transfer = '*'.join(temp)
            trip_chain_ex.append(on_station_name)
            trip_chain_ex.append(on_time)
            trip_chain_ex.append(get_time_bucket(on_time))
            trip_chain_ex.append(down_station_name)
            trip_chain_ex.append(down_time)
            trip_chain_ex.append(get_time_bucket(down_time))
            trip_chain_ex.append(line)
            trip_chain_ex.append(transfer)
            trip_chain_ex_array.append(trip_chain_ex)
            rownum = rownum + 1
            if (rownum == batch):
                cur.executemany(sql_insert, trip_chain_ex_array)
                totalnum = totalnum + rownum
                print(totalnum)
                rownum = 0
                trip_chain_ex_array = []
        if len(trip_chain_ex_array)!=0:
            cur.executemany(sql_insert, trip_chain_ex_array)
            totalnum = totalnum + rownum
            print(totalnum)
    except Exception as e:
        print(e.message)
        raise e
    finally:
        cur.close()
        connection.close()

def get_time_bucket(timestr):
    date = datetime.datetime.strptime(timestr, '%Y-%m-%d %H:%M:%S')
    hour_dep = str(date.time()).split(":")[0]
    minute_dep = "00"
    minute = int(str(date.time()).split(":")[1])
    if minute >= 30:
        minute_dep = "30"
    time_bucket = hour_dep + ":" + minute_dep
    return time_bucket

if __name__=="__main__":
    condition_array = get_group_condition()
    createTripChainExData(condition_array)
    print(get_time(),"################Complete Handle Chain####################")

# 三 通勤出行链处理(用sql脚本替代)

In [41]:
connection,cur = connect()
executeScriptsFromFile("c_TripChain2CommuterChain.sql",cur)
cur.close()
connection.close()

2019-09-18 17:00:28.744592 :  SQL: -- 根据出行链数据得到通勤链表   trip_chain_ex ---> commuter_trip_chain
-- 性能对比 基于4.8w的trip_chain_ex数据 python26min<->sql2s 
TRUNCATE TABLE commuter_trip_chain
2019-09-18 17:00:28.793564 :  Complete
2019-09-18 17:00:28.797559 :  SQL: 
INSERT INTO commuter_trip_chain (
SELECT CARD_ID,TRADE_DATE,
	case when substr(ON_TIME_BUCKET,1,2)<17 then substring_index(ON_STATION_NAME,',',1) ELSE '' end forward_on_station_name,
	case when substr(ON_TIME_BUCKET,1,2)<17 then substring_index(DOWN_STATION_NAME,',',1) ELSE '' end forward_down_station_name,
	case when substr(ON_TIME_BUCKET,1,2)<17 then substring_index(ON_TIME_BUCKET,',',1)  ELSE '' end forward_on_time_bucket,
	case when substr(ON_TIME_BUCKET,1,2)<17 then substring_index(LINE,',',1)  ELSE '' end forward_line,
	case when (substr(ON_TIME_BUCKET,1,2)<17 AND TRANSFER_STATION IS NOT NULL)then substring_index(TRANSFER_STATION,',',1)  ELSE '' end forward_transfer_station,
	case when (substr(ON_TIME_BUCKET,1,2)<17 AND LOCATE(',

In [ ]:
print(get_time(),"################Starting Commuter Chain####################")
import pymysql

batch = 1000

connection,cur = connect()

def get_group_condition():
    try:
        sql_condition = "select CARD_ID,TRADE_DATE,count(*) as CHAIN_NUM from trip_chain_ex group by CARD_ID,TRADE_DATE"
        cur.execute(sql_condition)
        results = cur.fetchall()
        print("总共数量＝" + str(len(results)))
        return results
    except Exception as e:
        print(e.message)
        raise e

def createCommuterTripChainData(condition_array):
    try:
        rownum = 0
        totalnum = 0
        commuter_trip_chain_array = []
        sql_insert = 'insert into commuter_trip_chain (CARD_ID,TRADE_DATE,\
                            FORWARD_ON_STATION_NAME,FORWARD_DOWN_STATION_NAME,FORWARD_ON_TIME_BUCKET,FORWARD_LINE,FORWARD_TRANSFER_STATION,\
                            BACK_ON_STATION_NAME,BACK_DOWN_STATION_NAME,BACK_ON_TIME_BUCKET,BACK_LINE,BACK_TRANSFER_STATION) \
                            values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        for condition in condition_array:
            commuter_trip_chain = []
            commuter_trip_chain.append(condition["CARD_ID"])
            commuter_trip_chain.append(condition["TRADE_DATE"])
            sql_condition = "select ON_STATION_NAME,ON_TIME_BUCKET,DOWN_STATION_NAME,DOWN_TIME_BUCKET,LINE,TRANSFER_STATION\
                                        from trip_chain_ex where CARD_ID = '%s' and TRADE_DATE = '%s' order by ON_TIME" % \
                                        (commuter_trip_chain[0],commuter_trip_chain[1])
            cur.execute(sql_condition)
            results = cur.fetchall()
            n = len(results)
            
            #原逻辑是出行首链的的上下车站为去链的上下车站，如果有>=2条链则末链的上下车站为回链的上下车站
            #现在改成：如果首链上车时间在下班时间17点前则出行首链的上下车站为去链的上下车站，
            #                       如果链数>=2则末链为回链  
            #               如果首链上车时间在下班时间17点后则末链的上下车站为回链的上下车站
            forward_on_station_name = ""
            forward_down_station_name = ""
            forward_on_time_bucket = ""
            forward_line = ""
            forward_transfer_station = ""
            back_on_station_name = ""
            back_down_station_name = ""
            back_on_time_bucket = ""
            back_line = ""
            back_transfer_station = ""
            first_chain_on_time = int(results[0]["ON_TIME_BUCKET"][0:2])

            if first_chain_on_time < 17 :
                forward_on_station_name = results[0]["ON_STATION_NAME"]
                forward_down_station_name = results[0]["DOWN_STATION_NAME"]
                forward_on_time_bucket = results[0]["ON_TIME_BUCKET"]
                forward_line = results[0]["LINE"]
                forward_transfer_station = results[0]["TRANSFER_STATION"]
                if n>=2:
                    back_on_station_name = results[n-1]["ON_STATION_NAME"]
                    back_down_station_name = results[n-1]["DOWN_STATION_NAME"]
                    back_on_time_bucket = results[n-1]["ON_TIME_BUCKET"]
                    back_line = results[n-1]["LINE"]
                    back_transfer_station = results[n-1]["TRANSFER_STATION"]
            if first_chain_on_time >= 17:
                back_on_station_name = results[n-1]["ON_STATION_NAME"]
                back_down_station_name = results[n-1]["DOWN_STATION_NAME"]
                back_on_time_bucket = results[n-1]["ON_TIME_BUCKET"]
                back_line = results[n-1]["LINE"]
                back_transfer_station = results[n-1]["TRANSFER_STATION"]
                
            commuter_trip_chain.append(forward_on_station_name)
            commuter_trip_chain.append(forward_down_station_name)
            commuter_trip_chain.append(forward_on_time_bucket)
            commuter_trip_chain.append(forward_line)
            commuter_trip_chain.append(forward_transfer_station)
            commuter_trip_chain.append(back_on_station_name)
            commuter_trip_chain.append(back_down_station_name)
            commuter_trip_chain.append(back_on_time_bucket)
            commuter_trip_chain.append(back_line)
            commuter_trip_chain.append(back_transfer_station)
            commuter_trip_chain_array.append(commuter_trip_chain)
            rownum = rownum + 1
            if (rownum == batch):
                cur.executemany(sql_insert, commuter_trip_chain_array)
                totalnum = totalnum + rownum
                print(totalnum)
                rownum = 0
                commuter_trip_chain_array = []
        if len(commuter_trip_chain_array)!=0:
            cur.executemany(sql_insert, commuter_trip_chain_array)
            totalnum = totalnum + rownum
            print(totalnum)
    except Exception as e:
        print(e.message)
        raise e
    finally:
        cur.close()
        connection.close()

if __name__=="__main__":
    condition_array = get_group_condition()
    createCommuterTripChainData(condition_array)
    print(get_time(),"################Complete Commuter Chain####################")

# 四 通勤出行链规则处理 

### 可以优化 详见0007乘客

In [42]:
import pymysql
import numpy as np
import pandas as pd

def station_bucket(forward_on_station_name_list,forward_down_station_name_list,back_on_station_name_list,back_down_station_name_list):
    list = []
    residence_station_name_list = []
    residence_station_name_list.extend(forward_on_station_name_list)
    residence_station_name_list.extend(back_down_station_name_list)
    dataframe_residence = pd.DataFrame(np.array(residence_station_name_list), columns=["station_name"])
    s_residence = dataframe_residence.groupby(["station_name"])["station_name"].count()
    residence_key = s_residence.idxmax()
    residence_value = s_residence[residence_key]
    occupation_station_name_list = []
    occupation_station_name_list.extend(forward_down_station_name_list)
    occupation_station_name_list.extend(back_on_station_name_list)
    dataframe_occupation = pd.DataFrame(np.array(occupation_station_name_list), columns=["station_name"])
    s_occupation = dataframe_occupation.groupby(["station_name"])["station_name"].count()
    occupation_key = s_occupation.idxmax()
    occupation_value = s_occupation[occupation_key]
    list.append(residence_key)
    list.append(occupation_key)
    list.append(residence_value+occupation_value)
    return list

def line_bucket(forward_line_list,back_line_list):
    list = []
    forward_line_key = ''
    forward_line_value = 0
    back_line_key = ''
    back_line_value = 0
    if len(forward_line_list)>0:
        dataframe_forward_line = pd.DataFrame(np.array(forward_line_list), columns=["line"])
        s_forward_line = dataframe_forward_line.groupby(["line"])["line"].count()
        forward_line_key = s_forward_line.idxmax()
        forward_line_value = s_forward_line[forward_line_key]
    if len(back_line_list) > 0:
        dataframe_back_line = pd.DataFrame(np.array(back_line_list), columns=["line"])
        s_back_line = dataframe_back_line.groupby(["line"])["line"].count()
        back_line_key = s_back_line.idxmax()
        back_line_value = s_back_line[back_line_key]
    list.append(forward_line_key)
    list.append(back_line_key)
    list.append(forward_line_value+back_line_value)
    return list

def line_bucket_ex(residence_station_name,occupation_station_name,forward_line,back_line,forward_on_station_name_list, forward_down_station_name_list, back_on_station_name_list,back_down_station_name_list,forward_line_list,back_line_list):
    num = 0
    forward_index = 0
    for forward_e in forward_line_list:
        if (forward_e != forward_line) and (forward_on_station_name_list[forward_index] == residence_station_name) and (forward_down_station_name_list[forward_index] == occupation_station_name):
            num = num + 1
    back_index = 0
    for back_e in back_line_list:
        if (back_e != back_line) and (back_on_station_name_list[back_index] == occupation_station_name) and (back_down_station_name_list[back_index] == residence_station_name):
            num = num + 1
    return num

def time_bucket(forward_time_list,back_time_list):
    list = []
    forward_time_key = ''
    forward_time_value = 0
    back_time_key = ''
    back_time_value = 0
    if len(forward_time_list)>0:
        dataframe_forward_time = pd.DataFrame(np.array(forward_time_list), columns=["time"])
        s_forward_time = dataframe_forward_time.groupby(["time"])["time"].count()
        forward_time_key = s_forward_time.idxmax()
        forward_time_value = s_forward_time[forward_time_key]
    if len(back_time_list) > 0:
        dataframe_back_time = pd.DataFrame(np.array(back_time_list), columns=["time"])
        s_back_time = dataframe_back_time.groupby(["time"])["time"].count()
        back_time_key = s_back_time.idxmax()
        back_time_value = s_back_time[back_time_key]
    list.append(forward_time_key)
    list.append(back_time_key)
    list.append(forward_time_value+back_time_value)
    return list

def transfer_bucket(forward_transfer_list,back_transfer_list):
    list = []
    forward_transfer_key = ''
    forward_transfer_value = 0
    back_transfer_key = ''
    back_transfer_value = 0
    if len(forward_transfer_list)>0:
        dataframe_forward_transfer = pd.DataFrame(np.array(forward_transfer_list), columns=["transfer"])
        s_forward_transfer = dataframe_forward_transfer.groupby(["transfer"])["transfer"].count()
        forward_transfer_key = s_forward_transfer.idxmax()
        forward_transfer_value = s_forward_transfer[forward_transfer_key]
    if len(back_transfer_list)>0:
        dataframe_back_transfer = pd.DataFrame(np.array(back_transfer_list), columns=["transfer"])
        s_back_transfer = dataframe_back_transfer.groupby(["transfer"])["transfer"].count()
        back_transfer_key = s_back_transfer.idxmax()
        back_transfer_value = s_back_transfer[back_transfer_key]
    list.append(forward_transfer_key)
    list.append(back_transfer_key)
    list.append(forward_transfer_value + back_transfer_value)
    return list


def createCommuterLawData():
    cur.execute("SELECT * FROM commuter_trip_chain t")
    results = cur.fetchall()
    results=pd.DataFrame(list(results),columns=[i[0] for i in cur.description])
    
    rownum = 0
    totalnum = 0
    cur.execute("TRUNCATE TABLE commuter_law ")
    sql_insert = "insert into commuter_law (CARD_ID,RESIDENCE_STATION_NAME,OCCUPATION_STATION_NAME,FORWARD_LINE,BACK_LINE,\
                                FORWARD_ON_TIME_BUCKET,BACK_ON_TIME_BUCKET,FORWARD_TRANSFER_STATION,BACK_TRANSFER_STATION,\
                                LAW_STATION_NUM,LAW_LINE_NUM,LAW_TIME_NUM,LAW_TRANSFER_NUM,DAY_NUM) \
                                values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"

    info_dict={}
    for card_id in set(results["CARD_ID"]):
        info_dict[card_id]={}
        info_dict[card_id]["FORWARD_ON_STATION_NAME_LIST"]=[]
        info_dict[card_id]["FORWARD_DOWN_STATION_NAME_LIST"]=[]
        info_dict[card_id]["BACK_ON_STATION_NAME_LIST"]=[]
        info_dict[card_id]["BACK_DOWN_STATION_NAME_LIST"]=[]
        info_dict[card_id]["FORWARD_LINE_LIST"]=[]
        info_dict[card_id]["FORWARD_ON_TIME_BUCKET_LIST"]=[]
        info_dict[card_id]["FORWARD_TRANSFER_STATION_LIST"]=[]
        info_dict[card_id]["BACK_LINE_LIST"]=[]
        info_dict[card_id]["BACK_ON_TIME_BUCKET_LIST"]=[]
        info_dict[card_id]["BACK_TRANSFER_STATION_LIST"]=[] 
        info_dict[card_id]["DAY_NUM"]=0
    for index,row in results.iterrows():
        card_id=row["CARD_ID"]
        if row["FORWARD_ON_STATION_NAME"] != '':
            info_dict[card_id]["FORWARD_ON_STATION_NAME_LIST"].append(row["FORWARD_ON_STATION_NAME"])
        if row["FORWARD_DOWN_STATION_NAME"] != '':
            info_dict[card_id]["FORWARD_DOWN_STATION_NAME_LIST"].append(row["FORWARD_DOWN_STATION_NAME"])
        if row["BACK_ON_STATION_NAME"] != '':
            info_dict[card_id]["BACK_ON_STATION_NAME_LIST"].append(row["BACK_ON_STATION_NAME"])
        if row["BACK_DOWN_STATION_NAME"] != '':
            info_dict[card_id]["BACK_DOWN_STATION_NAME_LIST"].append(row["BACK_DOWN_STATION_NAME"])
        if row["FORWARD_LINE"] != '':
            info_dict[card_id]["FORWARD_LINE_LIST"].append(row["FORWARD_LINE"])
        if row["FORWARD_ON_TIME_BUCKET"] != '':
            info_dict[card_id]["FORWARD_ON_TIME_BUCKET_LIST"].append(row["FORWARD_ON_TIME_BUCKET"])
        if row["FORWARD_TRANSFER_STATION"] != '':
            info_dict[card_id]["FORWARD_TRANSFER_STATION_LIST"].append(row["FORWARD_TRANSFER_STATION"])
        if row["BACK_LINE"] != '':
            info_dict[card_id]["BACK_LINE_LIST"].append(row["BACK_LINE"])
        if row["BACK_ON_TIME_BUCKET"] != '':
            info_dict[card_id]["BACK_ON_TIME_BUCKET_LIST"].append(row["BACK_ON_TIME_BUCKET"])
        if row["BACK_TRANSFER_STATION"] != '':
            info_dict[card_id]["BACK_TRANSFER_STATION_LIST"].append(row["BACK_TRANSFER_STATION"])
        info_dict[card_id]["DAY_NUM"]=info_dict[card_id]["DAY_NUM"]+1


    commuter_law_array=[]
    for cardid,cardinfo in info_dict.items():
        commuter_law=[]
        forward_on_station_name_list=cardinfo["FORWARD_ON_STATION_NAME_LIST"] 
        forward_down_station_name_list=cardinfo["FORWARD_DOWN_STATION_NAME_LIST"] 
        back_on_station_name_list=cardinfo["BACK_ON_STATION_NAME_LIST"] 
        back_down_station_name_list=cardinfo["BACK_DOWN_STATION_NAME_LIST"]
        forward_line_list=cardinfo["FORWARD_LINE_LIST"]
        back_line_list=cardinfo["BACK_LINE_LIST"]
        forward_time_list=cardinfo["FORWARD_ON_TIME_BUCKET_LIST"]
        back_time_list=cardinfo["BACK_ON_TIME_BUCKET_LIST"]
        forward_transfer_list=cardinfo["FORWARD_TRANSFER_STATION_LIST"]
        back_transfer_list=cardinfo["BACK_TRANSFER_STATION_LIST"]
        day_num = cardinfo["DAY_NUM"]
        res_station = station_bucket(forward_on_station_name_list, forward_down_station_name_list, back_on_station_name_list,back_down_station_name_list)
        residence_station_name = res_station[0]
        occupation_station_name = res_station[1]
        station_num = res_station[2]
        res_line = line_bucket(forward_line_list,back_line_list)
        forward_line = res_line[0]
        back_line = res_line[1]
        line_num = res_line[2]
        line_ex_num = line_bucket_ex(residence_station_name,occupation_station_name,forward_line,back_line,forward_on_station_name_list, forward_down_station_name_list, back_on_station_name_list,back_down_station_name_list,forward_line_list,back_line_list)
        if line_ex_num>0:
            line_num = line_num + line_ex_num
        res_time = time_bucket(forward_time_list,back_time_list)
        forward_time = res_time[0]
        back_time = res_time[1]
        time_num = res_time[2]
        res_transfer = time_bucket(forward_transfer_list,back_transfer_list)
        forward_transfer = res_transfer[0]
        back_transfer = res_transfer[1]
        transfer_num = res_transfer[2]
        commuter_law.append(cardid)
        commuter_law.append(residence_station_name)
        commuter_law.append(occupation_station_name)
        commuter_law.append(forward_line)
        commuter_law.append(back_line)
        commuter_law.append(forward_time)
        commuter_law.append(back_time)
        commuter_law.append(forward_transfer)
        commuter_law.append(back_transfer)
        commuter_law.append(int(station_num))
        commuter_law.append(int(line_num))
        commuter_law.append(int(time_num))
        commuter_law.append(int(transfer_num))
        commuter_law.append(int(day_num))
        #print(commuter_law)
        commuter_law_array.append(commuter_law)
        rownum = rownum + 1
        if (rownum == batch):
            cur.executemany(sql_insert, commuter_law_array)
            totalnum = totalnum + rownum
            print(totalnum)
            rownum = 0
            commuter_law_array = []
    if len(commuter_law_array)!=0:
        cur.executemany(sql_insert, commuter_law_array)
        totalnum = totalnum + len(commuter_law_array)
        print(totalnum)
        cur.executemany(sql_insert,commuter_law_array)
    cur.close()
    connection.close()
    
    
if __name__=="__main__":
    print(get_time(),"################Starting Analysis Law####################")
    batch = 1000
    connection,cur = connect1()
    createCommuterLawData()
    print(get_time(),"################Complete Analysis Law####################")

2019-09-18 17:01:42.161274 :  ################Starting Analysis Law####################
2019-09-18 17:01:42.231231 :  ################Starting Analysis Law####################
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
1

# 五 通勤规则打分

In [83]:
import pymysql
import numpy as np

batch = 1000

def get_result_data():
    try:
        sql_sample = "select CARD_ID,LAW_STATION_NUM,LAW_LINE_NUM,LAW_TIME_NUM,DAY_NUM from commuter_law limit 1000"
        cur.execute(sql_sample)
        results = cur.fetchall()
        print("总共数量＝" + str(len(results)))
        return results
    except Exception as e:
        print(e.message)
        raise e

def get_data_list(result_list,key_list):
    try:
        data_list = []
        for key in key_list:
            data_l = []
            for result_data in result_list:
                data_l.append(result_data[key])
            data_list.append(data_l)
        return data_list
    except Exception as e:
        print(e.message)
        raise e

def get_scale_list1(data_list):
    try:
        scale_list = []
        for data in data_list:
            mi = np.min(data)
            ma = np.max(data)
            data_l = []
            for v in data:
                data_l.append((v-mi)/(ma-mi))
            scale_list.append(data_l)
        return scale_list
    except Exception as e:
        print(e.message)
        raise e
def get_scale_list(data_list):
    try:
        scale_list = []
        
        for data in data_list:
            s = np.sum(data)
            data_l = []
            for v in data:
                data_l.append(v/s)
            scale_list.append(data_l)
        return scale_list
    except Exception as e:
        print(e.message)
        raise e

def get_feature_weight(scale_list):
    try:
        entropy_list = []

        k = -1/np.log(len(scale_list[0]))

        for data in scale_list:
            p = 0.0
            for v in data:
                if v==0:
                    p = p + 0.00001
                else:
                    logv = np.log2(v)
                    p = p + v * logv
            p = p * k
            entropy_list.append(p)
            
        #添加下面三行以修正书中直接把上面的list当作权重的错误    
        entropy_list =  [1-i for i in entropy_list]   
        entropy_sum = sum(entropy_list)
        weight_list = [i/entropy_sum for i in entropy_list]
        
        return weight_list

    except Exception as e:
        print(e.message)
        raise e

def get_data_preprocess_list(scale_list,weight_list):
    try:
        data_preprocess_list = []

        index = 0
        for data in scale_list:
            data_preprocess = []
            for v in data:
                if v==0:
                    data_preprocess.append(0.00001 * weight_list[index])
                else:
                    data_preprocess.append(v * weight_list[index])
            data_preprocess_list.append(data_preprocess)
            index = index + 1
        return data_preprocess_list

    except Exception as e:
        print(e.message)
        raise e

def get_ID():
    try:
        result_list = get_result_data()
        key_list = ["CARD_ID"]
        data_list = get_data_list(result_list,key_list)[0]
        return data_list
    except Exception as e:
        print(e.message)
        raise e

def get_data():
    try:
        result_list = get_result_data()
        key_list = ["LAW_STATION_NUM","LAW_LINE_NUM","LAW_TIME_NUM","DAY_NUM"]
        data_list = get_data_list(result_list,key_list)
        scale_list = get_scale_list(data_list)
        weight_list = get_feature_weight(scale_list)
        data_preprocess_list = get_data_preprocess_list(scale_list, weight_list)
        return data_preprocess_list
    except Exception as e:
        print(e.message)
        raise e

def prep_data_to_table(prep_table_list):
    cur.execute("Truncate table commuter_preprocessed_tmp")
    sql_insert = "insert into commuter_preprocessed_tmp  \
                            (CARD_ID,LAW_STATION_NUM_PREPED,LAW_LINE_NUM_PREPED,LAW_TIME_NUM_PREPED,DAY_NUM_PREPED) \
                         VALUES(%s,%s,%s,%s,%s)"    
    cur.executemany(sql_insert,prep_table_list)
    sql_update = "update commuter_law l  left join commuter_preprocessed_tmp p on l.card_id = p.card_id SET \
                                l.LAW_STATION_NUM_PREPED=p.LAW_STATION_NUM_PREPED, \
                                l.LAW_LINE_NUM_PREPED=p.LAW_LINE_NUM_PREPED,\
                                l.LAW_TIME_NUM_PREPED=p.LAW_TIME_NUM_PREPED,\
                                l.DAY_NUM_PREPED=p.DAY_NUM_PREPED \
                                WHERE 1=1"
    cur.execute(sql_update)
    cur.execute("COMMIT")
    return
        
def get_sample_array(data_preprocess_list):
    try:
        data_preprocess_array = np.array(data_preprocess_list)
        sample_list = data_preprocess_array.transpose()
        return sample_list
    except Exception as e:
        print(e.message)
        raise e

def get_goodandbad_sample(data_preprocess_list):
    try:
        goodandbad_list = []
        good_sample = []
        bad_sample = []
        for data in data_preprocess_list:
            good_sample.append(np.max(data))
            bad_sample.append(np.min(data))
        goodandbad_list.append(good_sample)
        goodandbad_list.append(bad_sample)
        goodandbad_array = np.array(goodandbad_list)
        return goodandbad_array
    except Exception as e:
        print(e.message)
        raise e

def get_distance(sample_array,maximum):
    try:
        distance_list = []
        good_sample = maximum[0]
        bad_sample = maximum[1]
        for sample in sample_array:
            distance = []
            dis1 = np.sqrt(np.sum(np.square(sample - good_sample)))
            dis2 = np.sqrt(np.sum(np.square(sample - bad_sample)))
            distance.append(dis1)
            distance.append(dis2)
            distance_list.append(distance)
        return distance_list
    except Exception as e:
        print(e.message)
        raise e

def get_score(distance_list):
    try:
        score_list = []
        for distance in distance_list:
            score = distance[1]/(distance[0] + distance[1])
            score_list.append(score)
        return score_list
    except Exception as e:
        print(e.message)
        raise e

        
def update_score(score_list):
    try:
        rownum = 0
        totalnum = 0
        data_list = []
        cur.execute("truncate table commuter_score_tmp")
        sql_insert = "insert into commuter_score_tmp(SCORE,CARD_ID) values(%s,%s)"
        for score in score_list:
            data_list.append(score)
            rownum = rownum + 1
            if (rownum == batch):
                cur.executemany(sql_insert, data_list)
                totalnum = totalnum + rownum
                print(totalnum)
                rownum = 0
                data_list = []
        if len(data_list)!=0:
            cur.executemany(sql_insert, data_list)
            totalnum = totalnum + len(data_list)
            print(totalnum)
        sql_update = "UPDATE commuter_law l LEFT JOIN commuter_score_tmp p ON  l.CARD_ID=p.CARD_ID SET  l.score = p.score where 1=1"
        cur.execute(sql_update)
        cur.execute("COMMIT")
    except Exception as e:
        print(e.message)
        raise e
    finally:
        cur.close()
        connection.close()

        

if __name__ == "__main__":
    print(get_time(),"################Starting Caculate Score####################")
    connection,cur = connect()
    id_list = get_ID()
    data_list = get_data()
    
    table_array = np.array([id_list]+data_list).transpose()
    prep_data_to_table(table_array.tolist())
    
    sample_array = get_sample_array(data_list)
    maximum = get_goodandbad_sample(data_list)
    distance_list = get_distance(sample_array,maximum)
    score_list = get_score(distance_list)
    temp_data = []
    temp_data.append(score_list)
    temp_data.append(id_list)
    temp_array = np.array(temp_data)
    score_array = temp_array.transpose()
    #update_score(score_array.tolist())
    update_score(score_array.tolist())
    print(get_time(),"################Complete Caculate Score####################")

2019-09-24 14:32:45.761484 :  ################Starting Caculate Score####################


# 六 通勤乘客聚类

In [182]:
import pymysql
import numpy as np
import pandas as pd


def get_sample_list(table_name, test=False):
    #取出带card_id的通勤特征数据【卡号，常起始站数量，常出行线路数量，常出行时段数量，出行天数】
    try:
        if test:
            sql_show = "select CARD_ID,LAW_STATION_NUM,LAW_LINE_NUM,LAW_TIME_NUM,DAY_NUM from %s limit 1000"%table_name
        else:
            sql_show = "select CARD_ID,LAW_STATION_NUM_PREPED,LAW_LINE_NUM_PREPED,LAW_TIME_NUM_PREPED,DAY_NUM_PREPED from %s"%table_name
        cur.execute(sql_show)
        results = cur.fetchall()
        result = results[0]
        key_list = [i[0] for i in cur.description]
        data_list = []
        for result_data in results:
            data_l = []
            for key in key_list:
                data_l.append(result_data[key])
            data_list.append(data_l)
        return data_list
    except Exception as e:
        print(e.message)
        raise e
    finally:
        cur.close()
        connection.close()

        
def get_scale_list(data_list):
    try:
        scale_list = []
        for data in data_list:
            s = np.sum(data)
            data_l = []
            for v in data:
                data_l.append(v/s)
            scale_list.append(data_l)
        return scale_list
    except Exception as e:
        print(e.message)
        raise e

def get_feature_weight(scale_list):
    try:
        entropy_list = []

        k = -1/np.log(len(scale_list[0]))

        for data in scale_list:
            p = 0.0
            for v in data:
                if v==0:
                    p = p + 0.00001
                else:
                    logv = np.log2(v)
                    p = p + v * logv
            p = p * k
            entropy_list.append(p)
            
        #添加下面三行以修正书中直接把上面的list当作权重的错误    
        entropy_list =  [1-i for i in entropy_list]   
        entropy_sum = sum(entropy_list)
        weight_list = [i/entropy_sum for i in entropy_list]
        
        return weight_list

    except Exception as e:
        print(e.message)
        raise e

def get_data_preprocess_list(scale_list,weight_list):
    try:
        data_preprocess_list = []

        index = 0
        for data in scale_list:
            data_preprocess = []
            for v in data:
                if v==0:
                    data_preprocess.append(0.00001 * weight_list[index])
                else:
                    data_preprocess.append(v * weight_list[index])
            data_preprocess_list.append(data_preprocess)
            index = index + 1
        return data_preprocess_list

    except Exception as e:
        print(e.message)
        raise e


def get_next_center_index(center_data_array,sample_data_array):
    #新质心生成法
    #对所有样本取离类质心最近的距离（应该是衡量此样本和所有类质心的最大相似度），
   # 然后取其中最大距离对应的样本索引（相当于选相似度最小的作为新质心）
    try:
        max_index = 0
        max_distance = 0

        index = 0
        if center_data_array[0][0] == str:
            center_data_array =  np.array([i[1:] for i in center_data_array])      
        
        for sample_data in sample_data_array:
            min_distance = -1
            for center_data in center_data_array:
                distance = np.sqrt(np.sum(np.square(center_data - sample_data)))
                if (min_distance == -1 or distance < min_distance):
                    min_distance = distance
            if min_distance > max_distance:
                max_distance = min_distance
                max_index = index
            index = index + 1

        return max_index
    except Exception as e:
        print(e.message)
        raise e

        
def get_center_array(sample_data_array,sample_label_array,k):
    #质心生成法
    #对任意样本集，先选取离远点最近的一个点作为质心，然后按新质心生成法生成指定数目的新质心
    #经过优化改为：
    #选取按指标排序后数量的k分点为k个质心，如果质心有重复删除重复质心然后按新质心生成法补全删掉的
    try:
        center_data_list = []
        center_label_list=[]
        index_list=[]
        sample_data_df=pd.DataFrame(sample_data_array)
        n=len(sample_data_array)
        
        for i in range(k):
            if i==0:
                intercept=0
            else:
                intercept =int(i*n/(k-1)-1)
            index = sample_data_df.sort_values(by=[0,1,2,3]).iloc[intercept,:].name
            index_list.append(index) 
            center_data_list.append(list(sample_data_array[index]))
            center_label_list.append(sample_label_array[index])
            
        center_data_array,center_label_array = np.array(center_data_list),np.array(center_label_list)
        
        #检查是否有重复的质心，如果有则删掉重复的，然后新质心生成法补全删掉的质心
        repeat_center_index_list=[]
        for i in range(len(center_data_array)-1):
            for j in range(i+1,len(center_data_array)):
                if (center_data_array[j] == center_data_array[i]).all():
                    repeat_center_index_list.append(j)
        
        if repeat_center_index_list != []:
            repeat_center_index_list=list(set(repeat_center_index_list))
            center_data_array=np.delete(center_data_array,repeat_center_index_list,axis=0)
            center_label_array=np.delete(center_label_array,repeat_center_index_list,axis=0)
            repeat_times=len(repeat_center_index_list)
            for k in range(repeat_times):
                next_center_index=get_next_center_index(center_data_array,sample_data_array)
                center_data_array = np.append(center_data_array, np.expand_dims(sample_data_array[next_center_index], axis=0), axis=0)
                center_label_array = np.append(center_label_array, np.expand_dims(sample_label_array[next_center_index], axis=0), axis=0)
            
        return center_data_array,center_label_array
    except Exception as e:
        print(e.message)
        raise e


def get_owner_center_index(center_data_array,sample_data):
    #对一个点，从A集合选取最近的点索引
    try:
        min_index = 0
        min_distance = -1

        index = 0
        for center_data in center_data_array:
            distance = np.sqrt(np.sum(np.square(center_data - sample_data)))
            if (min_distance == -1 or distance < min_distance):
                min_distance = distance
                min_index = index
            index = index + 1

        return min_index
    except Exception as e:
        print(e.message)
        raise e

        
def first_cluster(sample_data_array,sample_label_array,center_data_array):
    #对一个样本集和质心集，按距离归类
    try:
        
        group_data_dict = {}
        group_label_dict = {}
        for index,center_data in enumerate(center_data_array):
            group_data_dict[index] = []
            group_label_dict[index] = []

        for index,sample_data in enumerate(sample_data_array):
            
            center_index = get_owner_center_index(center_data_array, sample_data)
            group_data_dict[center_index].append(sample_data.tolist())
            group_label_dict[center_index].append(sample_label_array[index].tolist())
        
        for index in group_data_dict:
            group_data_dict[index] = np.array(group_data_dict[index])
            group_label_dict[index] = np.array(group_label_dict[index])

        return group_data_dict, group_label_dict
    except Exception as e:
        print(e.message)
        raise e

        
def split_center(split_data_array,center_array):    
    std = np.std(split_data_array,axis=0)
    max_std = std.max()
    max_std_index = std.argmax()
    center_high = center_array.copy()
    center_high[max_std_index] = center_high[max_std_index] +0.5*max_std
    center_low = center_array.copy()
    center_low[max_std_index] = center_low[max_std_index] -0.5*max_std
    splited_center_array = np.append(np.expand_dims(center_high,axis=0),np.expand_dims(center_low,axis=0),axis=0)
    return  splited_center_array

        
def get_center_data_array(group_data_dict):
    #对所有类组计算质心
    try:
        center_data_list = []
        center_data_array = np.array(center_data_list)

        for index in range(len(group_data_dict)):
            center_data = np.mean(group_data_dict[index], axis=0)
            if index == 0:
                center_data_array = np.expand_dims(center_data, axis=0)
            else:
                center_data_array = np.append(center_data_array, np.expand_dims(center_data, axis=0), axis=0)

        return center_data_array
    except Exception as e:
        print(e.message)
        raise e        
        
        
def cancel_cluster(group_data_dict, group_label_dict, center_data_array, min_n, k):
    #样本最少且小于指定值的组解散
    #对所有类组，选取最少样本数的组，如果样本数小于指定类样本数，则将此组解散并将组内样本按离其它各组质心的距离分到其它组
    try:
        new_group_data_dict = {}
        new_group_label_dict = {}
        new_center_data_array = np.array([])

        cluster_num = len(center_data_array)

        min_index = 0
        min_sample_num = 0

        for index in range(cluster_num):
            sample_num = len(group_data_dict[index])
            if (min_sample_num == 0 or min_sample_num > sample_num):
                min_sample_num = sample_num
                min_index = index
    

        if min_sample_num < min_n:
            new_center_data_array = np.delete(center_data_array, min_index, axis=0)
            new_group_data_dict, new_group_label_dict = first_cluster(sample_data_array, sample_label_array, new_center_data_array )
            new_center_data_array1 = get_center_data_array(new_group_data_dict)
            
        else:
            new_group_data_dict = group_data_dict
            new_group_label_dict = group_label_dict
            new_center_data_array = center_data_array
            new_center_data_array1 = get_center_data_array(new_group_data_dict)

        return new_group_data_dict, new_group_label_dict, new_center_data_array, new_center_data_array1
    except Exception as e:
        print(e.message)
        raise e

        
def split_cluster(group_data_dict, group_label_dict, center_data_array,min_n, k):
    #类内方差最大的一组分裂
    #对所有类组，选取样本数量大于2倍类样本下限中类组方差最大的组，按质心生成法生成两个质心，将此组用这两个质心分裂成两类
    #（这里的分裂与传统的ISODATA有所不同）
    #  1.传统的是指定分裂临界方差而不是不加判断直接对最大方差的组分裂）
    # 2.传统的是将要分裂组的质心，按照标准差最大分量的1/2，对质心相应位置进行+和-分离成两个质心
        # 比如质心为[5,8,7] 标准差[2,1,3] 那么新质心就是[5,8,7+1.5]和[5,8,7-1.5]
    try:
        cluster_num = len(center_data_array)

        max_var = 0
        max_index = 0
        
        for index in range(cluster_num):
            data_array = group_data_dict[index]
            data_var = np.var(data_array)
            if (index == 0 or max_var < data_var) and len(data_array)>2*min_n:
                max_var = data_var
                max_index = index
        

        split_data_array = group_data_dict[max_index]
        split_label_array = group_label_dict[max_index]
        
        new_center_data_array = center_data_array[max_index]        
        new_center_data_array = split_center(split_data_array,new_center_data_array)
        
        center_data_array = np.delete(center_data_array,max_index,0)
        new_center_data_array = np.append(center_data_array, new_center_data_array, axis=0)
        
        new_group_data_dict, new_group_label_dict = first_cluster(sample_data_array, sample_label_array, new_center_data_array )
        new_center_data_array1 = get_center_data_array(new_group_data_dict)
        
        return new_group_data_dict, new_group_label_dict, new_center_data_array, new_center_data_array1
    except Exception as e:
        print(e.message)
        raise e



def merge_cluster(group_data_dict, group_label_dict,center_data_array):
    # 类间距最小的两组合并
    #对所有类组，计算两两之间的质心距离，选取最小质心距离的两个组，进行合并，新质心为合并后的类组质心
    #（这里的分裂与传统的ISODATA有所不同）
    #  1.传统的是指定分裂临界合并类间距而不是不加判断直接对最最小距离的组合并）
    try:
        new_group_data_dict = {}
        new_group_label_dict = {}
        new_center_data_array = np.array([])
        
        cluster_num = len(center_data_array)
        a = 0
        b = 0
        min_distance = 0
        cluster_num = len(center_data_array)

        for x in range(cluster_num):
            for y in range(x):
                if y < x:
                    distance = np.sqrt(np.sum(np.square(center_data_array[x] - center_data_array[y])))
                    if (min_distance == 0 or min_distance > distance):
                        min_distance = distance
                        a = x
                        b = y
        
        if cluster_num > 2:
            
            center_data_array = np.delete(center_data_array,[a,b],0)
            merge_data_array = np.append(group_data_dict[a], group_data_dict[b], axis=0)
            merge_label_array = np.append(group_label_dict[a], group_label_dict[b], axis=0)
            merge_center_data = np.expand_dims(np.mean(merge_data_array, axis=0),axis=0)
            new_center_data_array = np.append(center_data_array, merge_center_data, axis=0)
            
            new_group_data_dict, new_group_label_dict = first_cluster(sample_data_array, sample_label_array, new_center_data_array )
            new_center_data_array1 = get_center_data_array(new_group_data_dict)
            
        else:
            new_group_data_dict = group_data_dict
            new_group_label_dict = group_label_dict
            new_center_data_array = center_data_array
            new_center_data_array1 = get_center_data_array(new_group_data_dict)
            
        return new_group_data_dict, new_group_label_dict, new_center_data_array, new_center_data_array1
    except Exception as e:
        print(e.message)
        raise e

        
def iteration_cluster(group_data_dict, group_label_dict,center_data_array,min_n,k):
    #对类组进行解散--分裂--合并的循环迭代
    try:
        
        new_group_data_dict, new_group_label_dict, new_center_data_array, new_center_data_array1= cancel_cluster(group_data_dict,group_label_dict,center_data_array, min_n,k)
        
        print("解散后：")
        for x in range(len(new_group_data_dict)):
            print(new_center_data_array[x],len(new_group_data_dict[x]),"----",new_center_data_array1[x])
        
        
        new_group_data_dict, new_group_label_dict, new_center_data_array, new_center_data_array1= split_cluster(new_group_data_dict,new_group_label_dict,new_center_data_array1,min_n,k)
        
        print("拆分后：")
        for x in range(len(new_group_data_dict)):
            print(new_center_data_array[x],len(new_group_data_dict[x]),"----",new_center_data_array1[x])
        
        new_group_data_dict, new_group_label_dict, new_center_data_array, new_center_data_array1 = merge_cluster(new_group_data_dict,new_group_label_dict,new_center_data_array1)
        
        print("合并后：")
        for x in range(len(new_group_data_dict)):
            print(new_center_data_array[x],len(new_group_data_dict[x]),"----",new_center_data_array1[x])
            
        return new_group_data_dict, new_group_label_dict, new_center_data_array, new_center_data_array1
    except Exception as e:
        print(e.message)
        raise e

        
def main_cluster(c,k,min_rate,iteration_times,sample_array,test):
    try:
        print(sample_array[0])

        global sample_data_array
        global sample_label_array
        
        sample_data_array = sample_array[:, 1:].astype(float)

        sample_label_array = sample_array[:, 0:1]
        
        ###归一加权处理
        if test:
            scale_list = get_scale_list(sample_data_array.transpose().tolist())
            weight_list = get_feature_weight(scale_list)
            data_preprocess_list = get_data_preprocess_list(scale_list, weight_list)
            sample_data_array = np.array(data_preprocess_list).astype(float).transpose()
            print(sample_data_array[0])

        min_n = round(len(sample_list) * min_rate)

        center_data_array, center_label_array = get_center_array(sample_data_array, sample_label_array, c)
        group_data_dict, group_label_dict = first_cluster(sample_data_array, sample_label_array, center_data_array)
        #center_data_array = get_center_data_array(group_data_dict)  
        #这一步有没有更新质心并没有影响测试结果（因为之后的iteration_cluster中的第一个cancel会更新消弭了这种影响）
        center_data_array_list = []

        for index in range(iteration_times):
            print("\n",get_time(),"################第 %s 次迭代####################"%index)
            group_data_dict, group_label_dict, center_data_array,center_data_array1 = iteration_cluster(group_data_dict, group_label_dict, center_data_array, min_n,k)
            center_data_array_list.append(np.sort(center_data_array1,axis=0))

            if index>=1:
                for i in range(index):
                    if len(center_data_array_list[index])==len(center_data_array_list[i]):
                        if (center_data_array_list[index]==center_data_array_list[i]).all() :
                            print("迭代中止：第%s次迭代质心集在第%s迭代结果中出现过！！！"%(index,i))
                            return group_data_dict, group_label_dict, center_data_array1
        return group_data_dict, group_label_dict, center_data_array1
    except Exception as e:
        print(e.message)
        raise e


def update_cluster(group_label_dict,table_name):
    connection,cur = connect()
    try:
        cur.execute("TRUNCATE table commuter_cluster_tmp")
        total = 0
        cluster_sql = "INSERT INTO commuter_cluster_tmp(CARD_ID,CLUSTER) VALUES(%s,%s)" 
        for class_ in group_label_dict:
            group_class_list = np.c_[group_label_dict[class_],[class_]*len(group_label_dict[class_])].tolist()
            batch_label_list = []
            index = 0
            for label_class in group_class_list:
                batch_label_list.append(label_class)
                index = index+1
                total = total +1
                if index%1000 == 0:
                    print(index)
                    cur.executemany(cluster_sql, batch_label_list)  
                    batch_label_list = []                    
            if len(batch_label_list) != 0:
                cur.executemany(cluster_sql, batch_label_list)  
        print("总个数：",total)
        sql_update = "UPDATE %s l LEFT JOIN commuter_cluster_tmp p ON  l.CARD_ID=p.CARD_ID SET  l.cluster = p.cluster where 1=1" %table_name
        print(sql_update)
        cur.execute(sql_update)
        cur.execute("COMMIT")
    except Exception as e:
        print(e.message)
        raise e
    finally:
        cur.close()
        connection.close()
        
def main(table_name, test, C, K, min_rate, iteration_times):   
    #table_name 选择聚类的数据对象
    #test 是选取所有数据对象还是抽取部分做测试
    #C 初始聚类数目
    #K 指定聚类数目
    #min_rate 最小簇样本数占比
    #iteration_times 迭代次数
    global connection,cur 
    connection,cur = connect()
    
    
    print(get_time(),"################Starting Cluster####################")
    sample_array = np.array(get_sample_list(table_name,test))
    group_data_dict, group_label_dict, center_data_array1=main_cluster(C,K,min_rate,iteration_times,sample_array,test) 
    
    group_data_dict_sorted = {}
    group_label_dict_sorted = {}
    sort_index = np.argsort(center_data_array1[:,0])
    for i,j in enumerate(sort_index):
        group_data_dict_sorted[i] = group_data_dict.pop(j)
        group_label_dict_sorted[i] = group_label_dict.pop(j)

    
    print(get_time(),"################Complete Cluster####################")
    
    update_cluster(group_label_dict_sorted,table_name)
    
    print(get_time(),"################Complete Update Cluster####################")
    return group_data_dict_sorted, group_label_dict_sorted, center_data_array1



"""
if __name__ == "__main__":
    #选择聚类的数据对象
    table_name = "commuter_law1"    
    #是选取所有数据对象还是抽取部分做测试
    test = False
    #初始聚类数目
    C = 6
    #指定聚类数目
    K = 3
    #最小簇样本数占比
    min_rate = 0.01
    #迭代次数
    iteration_times = 3
    
    group_data_dict, group_label_dict, center_data_array=main_cluster(C,K,min_rate,iteration_times,test)
    #update_cluster(group_label_dict)
    print(get_time(),"################Complete Cluster####################")
"""

'\nif __name__ == "__main__":\n    #选择聚类的数据对象\n    table_name = "commuter_law1"    \n    #是选取所有数据对象还是抽取部分做测试\n    test = False\n    #初始聚类数目\n    C = 6\n    #指定聚类数目\n    K = 3\n    #最小簇样本数占比\n    min_rate = 0.01\n    #迭代次数\n    iteration_times = 3\n    \n    group_data_dict, group_label_dict, center_data_array=main_cluster(C,K,min_rate,iteration_times,test)\n    #update_cluster(group_label_dict)\n    print(get_time(),"################Complete Cluster####################")\n'

# 修改车站名之后->使用归一加权后的数据聚类的测试

In [183]:
group_data_dict1, group_label_dict1, center_data_array1=main(table_name = "commuter_law1",test = True,C = 6,K = 4,min_rate = 0.01,iteration_times = 50)

2019-09-27 20:48:54.565751 :  ################Starting Cluster####################
['9000000151458272' '2' '1' '1' '1']
[1.17046251e-04 9.34267146e-05 1.24752919e-04 1.10685953e-04]

 2019-09-27 20:48:55.314293 :  ################第 0 次迭代####################
解散后：
[1.17046251e-04 9.34267146e-05 1.24752919e-04 1.10685953e-04] 561 ---- [1.17046251e-04 9.34267146e-05 1.24752919e-04 1.13842772e-04]
[0.00011705 0.00018685 0.00024951 0.00022137] 202 ---- [0.00017673 0.00019148 0.00023468 0.00018411]
[0.00023409 0.00037371 0.00024951 0.00033206] 185 ---- [0.00041725 0.00044744 0.00037156 0.00048462]
[0.00128751 0.00093427 0.00149704 0.00143892] 42 ---- [0.00122063 0.00113002 0.00114654 0.0011543 ]
[0.00140456 0.00214881 0.00237031 0.00132823] 10 ---- [0.00189615 0.00202736 0.00220813 0.00140571]
拆分后：
[1.17046251e-04 9.34267146e-05 1.24752919e-04 1.13842772e-04] 561 ---- [1.17046251e-04 9.34267146e-05 1.24752919e-04 1.13842772e-04]
[0.00017673 0.00019148 0.00023468 0.00018411] 254 ---- [0.000194

合并后：
[0.00053292 0.00055725 0.00045596 0.00061906] 120 ---- [0.00052134 0.00054655 0.00044807 0.00060508]
[0.00196318 0.00197895 0.00214348 0.00138861] 12 ---- [0.00188249 0.00192303 0.00216238 0.0013928 ]
[0.00014561 0.00013614 0.00016456 0.00015061] 833 ---- [0.00014402 0.00013414 0.00016324 0.00014869]
[0.00124037 0.00118081 0.00119555 0.00115298] 35 ---- [0.00124738 0.00117718 0.00116198 0.00114481]

 2019-09-27 20:48:58.766160 :  ################第 6 次迭代####################
解散后：
[0.00053292 0.00055725 0.00045596 0.00061906] 120 ---- [0.00052134 0.00054655 0.00044807 0.00060508]
[0.00196318 0.00197895 0.00214348 0.00138861] 12 ---- [0.00188249 0.00192303 0.00216238 0.0013928 ]
[0.00014561 0.00013614 0.00016456 0.00015061] 833 ---- [0.00014402 0.00013414 0.00016324 0.00014869]
[0.00124037 0.00118081 0.00119555 0.00115298] 35 ---- [0.00124738 0.00117718 0.00116198 0.00114481]
拆分后：
[0.00052134 0.00054655 0.00044807 0.00060508] 118 ---- [0.00050588 0.00053602 0.00043769 0.00058438]
[0.0

合并后：
[0.00185473 0.00189728 0.0021208  0.00141337] 14 ---- [0.00183094 0.00186853 0.00209407 0.00139939]
[0.00014551 0.00013585 0.00016431 0.00015026] 831 ---- [0.00014381 0.00013356 0.00016288 0.00014798]
[0.00052671 0.00055312 0.00045375 0.00061416] 121 ---- [0.00051365 0.00054357 0.00044127 0.00059917]
[0.00123066 0.00115582 0.00113347 0.00112583] 34 ---- [0.00122038 0.00113761 0.00112645 0.00112314]

 2019-09-27 20:49:01.124709 :  ################第 12 次迭代####################
解散后：
[0.00185473 0.00189728 0.0021208  0.00141337] 14 ---- [0.00183094 0.00186853 0.00209407 0.00139939]
[0.00014551 0.00013585 0.00016431 0.00015026] 831 ---- [0.00014381 0.00013356 0.00016288 0.00014798]
[0.00052671 0.00055312 0.00045375 0.00061416] 121 ---- [0.00051365 0.00054357 0.00044127 0.00059917]
[0.00123066 0.00115582 0.00113347 0.00112583] 34 ---- [0.00122038 0.00113761 0.00112645 0.00112314]
拆分后：
[0.00183094 0.00186853 0.00209407 0.00139939] 14 ---- [0.00183094 0.00186853 0.00209407 0.00139939]
[0.0

合并后：
[0.00180121 0.00180625 0.00193367 0.00136513] 19 ---- [0.0017865  0.00178494 0.00191069 0.00135736]
[0.00014381 0.00013356 0.00016288 0.00014798] 825 ---- [0.00014294 0.00013227 0.00016165 0.00014597]
[0.00049617 0.000524   0.00043501 0.00057845] 121 ---- [0.00048463 0.00051346 0.00042993 0.00057081]
[0.00110544 0.00105105 0.00100495 0.00108841] 35 ---- [0.00109355 0.00104104 0.00099089 0.00108472]

 2019-09-27 20:49:03.727097 :  ################第 18 次迭代####################
解散后：
[0.00180121 0.00180625 0.00193367 0.00136513] 19 ---- [0.0017865  0.00178494 0.00191069 0.00135736]
[0.00014381 0.00013356 0.00016288 0.00014798] 825 ---- [0.00014294 0.00013227 0.00016165 0.00014597]
[0.00049617 0.000524   0.00043501 0.00057845] 121 ---- [0.00048463 0.00051346 0.00042993 0.00057081]
[0.00110544 0.00105105 0.00100495 0.00108841] 35 ---- [0.00109355 0.00104104 0.00099089 0.00108472]
拆分后：
[0.0017865  0.00178494 0.00191069 0.00135736] 19 ---- [0.0017865  0.00178494 0.00191069 0.00135736]
[0.0

In [39]:
group_data_dict1, group_label_dict1, center_data_array1=main(table_name = "commuter_law",test = False,C = 6,K = 4,min_rate = 0.01,iteration_times = 20)

2019-09-23 19:06:52.623668 :  ################Starting Cluster####################

 2019-09-23 19:16:30.093181 :  ################第 0 次迭代####################
解散后：
[1.36269564e-07 1.08966798e-07 1.46274634e-07 1.29437831e-07] 471354 ---- [1.36464275e-07 1.08966798e-07 1.46274634e-07 1.33538011e-07]
[1.36269564e-07 2.17933596e-07 2.92549267e-07 2.58875661e-07] 198542 ---- [2.14527685e-07 2.33974945e-07 2.73581056e-07 2.20817254e-07]
[2.72539128e-07 4.35867191e-07 2.92549267e-07 5.17751323e-07] 154137 ---- [5.84166563e-07 6.28996820e-07 5.30109248e-07 6.77008972e-07]
[2.65725650e-06 3.05107034e-06 8.77647802e-07 1.81212963e-06] 12087 ---- [2.13231935e-06 2.11003607e-06 1.58660841e-06 1.54562927e-06]
[1.83963912e-06 8.71734382e-07 3.07176731e-06 1.81212963e-06] 14886 ---- [1.75218437e-06 1.69418263e-06 2.00455010e-06 1.47016194e-06]
拆分后：
[1.36464275e-07 1.08966798e-07 1.46274634e-07 1.33538011e-07] 472199 ---- [1.36463926e-07 1.08966798e-07 1.46274634e-07 1.33993931e-07]
[2.14527685e-07 2

解散后：
[1.21838422e-06 1.24560334e-06 1.07599456e-06 1.23680552e-06] 52089 ---- [1.02471107e-06 1.06184315e-06 9.03002746e-07 1.08920957e-06]
[1.70327712e-06 1.84972930e-06 1.89413668e-06 1.51474544e-06] 20358 ---- [2.07358259e-06 2.02535974e-06 1.98642648e-06 1.54100194e-06]
[2.13151732e-07 2.09133354e-07 2.27268875e-07 2.29231277e-07] 778559 ---- [1.96804109e-07 1.91313002e-07 2.13875636e-07 2.10127432e-07]
拆分后：
[1.02471107e-06 1.06184315e-06 9.03002746e-07 1.08920957e-06] 63887 ---- [9.26356602e-07 9.68697043e-07 8.29305532e-07 1.01100778e-06]
[1.96804109e-07 1.91313002e-07 2.13875636e-07 2.10127432e-07] 765317 ---- [1.89312556e-07 1.82837185e-07 2.07852918e-07 2.00827475e-07]
[2.07358259e-06 2.02535974e-06 2.26638636e-06 1.54100194e-06] 8997 ---- [2.29381798e-06 2.21678459e-06 2.48471779e-06 1.58832893e-06]
[2.07358259e-06 2.02535974e-06 1.70646661e-06 1.54100194e-06] 12805 ---- [1.88286120e-06 1.85326951e-06 1.52914965e-06 1.49371560e-06]
合并后：
[9.26356602e-07 9.68697043e-07 8.293055

拆分后：
[1.85615446e-07 1.78554314e-07 2.04824906e-07 1.96214353e-07] 754266 ---- [1.84016919e-07 1.76630963e-07 2.03562652e-07 1.94110584e-07]
[9.00545702e-07 9.43553715e-07 8.04314353e-07 9.82757375e-07] 72730 ---- [8.52883903e-07 8.98112348e-07 7.64391052e-07 9.46065429e-07]
[2.08327036e-06 2.02961168e-06 2.25535582e-06 1.54037464e-06] 9034 ---- [2.28908431e-06 2.21214420e-06 2.48313901e-06 1.58801334e-06]
[2.08327036e-06 2.02961168e-06 1.67854770e-06 1.54037464e-06] 14976 ---- [1.82333336e-06 1.80022230e-06 1.49731270e-06 1.47834494e-06]
合并后：
[1.84016919e-07 1.76630963e-07 2.03562652e-07 1.94110584e-07] 749440 ---- [1.81947524e-07 1.74127326e-07 2.01945420e-07 1.91356727e-07]
[8.52883903e-07 8.98112348e-07 7.64391052e-07 9.46065429e-07] 76579 ---- [8.24878487e-07 8.71801260e-07 7.35019576e-07 9.20653382e-07]
[1.99857676e-06 1.95521199e-06 1.86823960e-06 1.51960876e-06] 24987 ---- [1.97249199e-06 1.93026089e-06 1.85528243e-06 1.51242876e-06]

 2019-09-23 20:15:34.322274 :  ############

拆分后：
[1.81947524e-07 1.74127326e-07 2.01945420e-07 1.91356727e-07] 745803 ---- [1.80471575e-07 1.72453509e-07 2.00641782e-07 1.89383550e-07]
[8.25024345e-07 8.71927853e-07 7.35184811e-07 9.20717357e-07] 77623 ---- [7.89668457e-07 8.36452796e-07 7.08424655e-07 8.91238903e-07]
[1.97291782e-06 1.93067798e-06 2.14950960e-06 1.51268282e-06] 11411 ---- [2.20832883e-06 2.13696581e-06 2.36251928e-06 1.56939541e-06]
[1.97291782e-06 1.93067798e-06 1.56174641e-06 1.51268282e-06] 16169 ---- [1.71450693e-06 1.70461030e-06 1.38555162e-06 1.44550252e-06]
合并后：
[1.80471575e-07 1.72453509e-07 2.00641782e-07 1.89383550e-07] 742543 ---- [1.79241724e-07 1.70851072e-07 1.99601996e-07 1.87683573e-07]
[7.89668457e-07 8.36452796e-07 7.08424655e-07 8.91238903e-07] 80652 ---- [7.75945952e-07 8.24217119e-07 6.93154087e-07 8.77438338e-07]
[1.91882171e-06 1.88349387e-06 1.78976402e-06 1.49676219e-06] 27811 ---- [1.91066481e-06 1.87523118e-06 1.79330671e-06 1.49487174e-06]

 2019-09-23 20:47:23.529690 :  ###########

405000
406000
407000
408000
409000
410000
411000
412000
413000
414000
415000
416000
417000
418000
419000
420000
421000
422000
423000
424000
425000
426000
427000
428000
429000
430000
431000
432000
433000
434000
435000
436000
437000
438000
439000
440000
441000
442000
443000
444000
445000
446000
447000
448000
449000
450000
451000
452000
453000
454000
455000
456000
457000
458000
459000
460000
461000
462000
463000
464000
465000
466000
467000
468000
469000
470000
471000
472000
473000
474000
475000
476000
477000
478000
479000
480000
481000
482000
483000
484000
485000
486000
487000
488000
489000
490000
491000
492000
493000
494000
495000
496000
497000
498000
499000
500000
501000
502000
503000
504000
505000
506000
507000
508000
509000
510000
511000
512000
513000
514000
515000
516000
517000
518000
519000
520000
521000
522000
523000
524000
525000
526000
527000
528000
529000
530000
531000
532000
533000
534000
535000
536000
537000
538000
539000
540000
541000
542000
543000
544000
545000
546000
547000

In [118]:
group_data_dict2, group_label_dict2, center_data_array2=main(table_name = "commuter_law2",test = True,C = 6,K = 4,min_rate = 0.01,iteration_times = 50)

2019-09-24 16:43:47.913661 :  ################Starting Cluster####################
['6226378665256837712' '2' '1' '1' '1']
[1.17361631 0.96870799 1.19868487 1.08350088]

 2019-09-24 16:47:49.438562 :  ################第 0 次迭代####################
解散后：
[1.17361631 0.96870799 1.19868487 1.08350088] 155415 ---- [1.17534938 0.96870799 1.19868487 1.1177666 ]
[1.17361631 1.93741599 2.39736975 2.16700176] 64700 ---- [1.84170692 2.06123693 2.23887285 1.84466443]
[2.34723262 3.87483198 2.39736975 4.33400351] 50413 ---- [4.87347706 5.44173585 4.19726358 5.52924384]
[18.77786099 29.06123983  7.19210924 16.25251317] 2359 ---- [18.82935285 19.69281924 12.25768343 13.29596584]
[14.6702039  10.65578794 23.97369748 14.08551141] 7527 ---- [15.10537902 15.08556463 15.15786152 12.13313697]
拆分后：
[1.17534938 0.96870799 1.19868487 1.1177666 ] 155705 ---- [1.17534615 0.96870799 1.19868487 1.12173881]
[1.84170692 2.06123693 2.23887285 1.84466443] 80470 ---- [2.03384046 2.30052198 2.31121101 2.14998243]
[4.87347

拆分后：
[4.30676299 4.85491057 3.79082276 5.03058889] 35084 ---- [4.20245896 4.7414065  3.73654579 4.90274573]
[19.2509049  19.17028109 17.29948937 13.14564526] 5027 ---- [19.37062244 19.27318456 17.305223   13.1651715 ]
[1.40596763 1.34284682 1.52826807 1.39023316] 226055 ---- [1.39908013 1.33336781 1.52101553 1.38291532]
[10.27607456 10.897821   10.26612364 10.17379955] 6020 ---- [11.23240489 11.55626799 11.58622516 10.78623324]
[10.27607456 10.897821    7.34831013 10.17379955] 8228 ---- [ 9.34920612 10.21334693  6.57339088  9.57307527]
合并后：
[4.20245896 4.7414065  3.73654579 4.90274573] 35847 ---- [4.1934512  4.72744742 3.7234569  4.885804  ]
[19.37062244 19.27318456 17.305223   13.1651715 ] 5276 ---- [19.11797838 19.05027799 17.17440972 13.12194315]
[1.39908013 1.33336781 1.52101553 1.38291532] 225446 ---- [1.39536059 1.32908493 1.51661715 1.37845672]
[10.14488668 10.78075181  8.69139077 10.08565324] 13845 ---- [10.10191618 10.73611129  8.64949187 10.06603228]

 2019-09-24 17:01:45.789

拆分后：
[7.41209938 8.04863691 6.30058174 7.97412832] 24730 ---- [7.05194862 7.68812573 6.01756287 7.68199055]
[1.58512916 1.57012409 1.66933226 1.62834624] 247135 ---- [1.5670189  1.54762014 1.65629356 1.60483169]
[17.30829981 17.45289529 17.89080006 12.7422336 ] 3765 ---- [19.04983449 19.03144567 19.35438305 13.14762845]
[17.30829981 17.45289529 13.05221891 12.7422336 ] 4784 ---- [15.10160112 15.46834707 11.44288033 12.21112737]
合并后：
[7.05194862 7.68812573 6.01756287 7.68199055] 25864 ---- [6.88623818 7.53164473 5.84326434 7.53105165]
[1.5670189  1.54762014 1.65629356 1.60483169] 245846 ---- [1.55366461 1.53005159 1.64628406 1.587045  ]
[16.84041252 17.0375442  14.92712501 12.62356468] 8704 ---- [16.72342568 16.92289684 14.92324107 12.58648595]

 2019-09-24 17:13:19.373633 :  ################第 12 次迭代####################
解散后：
[7.05194862 7.68812573 6.01756287 7.68199055] 25864 ---- [6.88623818 7.53164473 5.84326434 7.53105165]
[1.5670189  1.54762014 1.65629356 1.60483169] 245846 ---- [1.

合并后：
[18.9965001  18.97281242 19.335955   13.13562419] 3527 ---- [20.62346997 20.30029637 19.12593988 13.39983691]
[14.40133226 14.81118091 11.09186022 11.98254463] 14127 ---- [11.14104735 11.74381015  9.45813021 10.57945382]
[2.02428229 2.06223416 2.01893092 2.11840912] 262760 ---- [1.80963631 1.82961054 1.8445815  1.8944565 ]

 2019-09-24 17:23:44.585665 :  ################第 19 次迭代####################
解散后：
[18.9965001  18.97281242 19.335955   13.13562419] 3527 ---- [20.62346997 20.30029637 19.12593988 13.39983691]
[14.40133226 14.81118091 11.09186022 11.98254463] 14127 ---- [11.14104735 11.74381015  9.45813021 10.57945382]
[2.02428229 2.06223416 2.01893092 2.11840912] 262760 ---- [1.80963631 1.82961054 1.8445815  1.8944565 ]
拆分后：
[20.62346997 20.30029637 19.12593988 13.39983691] 4073 ---- [20.31165908 20.04271824 18.16568226 13.32336311]
[1.80963631 1.82961054 1.8445815  1.8944565 ] 257105 ---- [1.70191825 1.70693127 1.76122127 1.77030739]
[11.14104735 11.74381015 11.05398617 10.5794

2019-09-24 17:35:32.603556 :  ################Complete Cluster####################
1000
2000
3000
4000
5000
6000
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
1

# 通勤指标分析

In [89]:
def law_analysis(character,top_k):
    sql_show =  "SELECT a.cluster,a.%s,a.num,concat(truncate(a.num/b.total_num*100,2),\"%s\") FROM ( \
                          SELECT a.cluster,a.%s,a.num FROM  \
                          (SELECT l.CLUSTER,l.%s,COUNT(*) num FROM commuter_law l GROUP BY l.CLUSTER,l.%s) a  \
                          WHERE (SELECT COUNT(*)   FROM (\
                          SELECT l.CLUSTER,l.%s,COUNT(*) num FROM commuter_law l GROUP BY l.CLUSTER,l.%s) b   \
                          WHERE cluster=a.cluster AND num>a.num  )<%s   ) a \
                          LEFT JOIN (SELECT cluster,COUNT(*) total_num FROM commuter_law b GROUP BY cluster) b \
                          ON a.cluster=b.cluster ORDER BY a.cluster,a.num desc "   %(character,"%",character,character,character,character,character,top_k)
    connection,cur = connect()
    cur.execute(sql_show)
    results = cur.fetchall()
    result = results[0]
    key_list = [i[0] for i in cur.description]
    data_list = []
    for result_data in results:
        data_l = []
        for key in key_list:
            data_l.append(result_data[key])
        data_list.append(data_l)
    key_list_CN = ["乘客类别",character_dict[character],"数量","同类比例"]
    data_frame = pd.DataFrame(data_list,columns=key_list_CN)
    cur.close()
    connection.close()
    return data_frame

character_dict={"RESIDENCE_STATION_NAME":"最频住站","OCCUPATION_STATION_NAME":"最频职站","FORWARD_LINE":"最频去程线路过程",
                            "FORWARD_ON_TIME_BUCKET":"最频去程时段","FORWARD_TRANSFER_STATION":"最频去程换乘过程","BACK_LINE":"最频回程线路过程",
                            "BACK_ON_TIME_BUCKET":"最频回程时段","BACK_TRANSFER_STATION":"最频回程换乘过程"}

if __name__ == "__main__":
    index=1
    top_k=3
    for i in character_dict:
        df=law_analysis(i,top_k)
        df.to_excel(base_dir+'\data_analyse\\result\通勤指标-%s-统计分析1.xlsx'% \
                    (character_dict[i])  , header=True)

In [122]:
df.to_excel(base_dir+'\data_analyse\\result\通勤指标-%s-统计分析1.xlsx'% \
                (character_dict[i])  , header=True)

In [74]:
pd.DataFrame(data_list,columns=key_list)

,cluster,FORWARD_LINE,num,"concat(truncate(a.num/b.total_num*100,2),""%"")"
0,0,201150,2041,13.12%
1,0,202020,1022,6.57%
2,0,200960,935,6.01%
3,1,,148678,19.26%
4,1,202020,75274,9.75%
5,1,200960,65393,8.47%
6,2,201150,6809,10.69%
7,2,200960,5884,9.23%
8,2,202020,5527,8.67%


# 使用iris数据测试聚类性能

In [45]:
from sklearn.datasets import load_iris
iris=load_iris()
iris_data=iris.data.astype(np.float32)
iris_label=iris.target
iris_label=np.expand_dims(iris_label,axis=1)
iris=pd.DataFrame(iris_data)
iris.ix[:,"label"]=iris_label
global sample_data_array
global sample_label_array
iris_id=np.expand_dims(np.arange(150),axis=1)  
sample_data_array = iris_data
sample_label_array=iris_id


center_data_array, center_label_array = get_center_array(iris_data, iris_id, 3)
group_data_dict, group_label_dict = first_cluster(iris_data, iris_id, center_data_array)


for index in range(3):
    print("\n",get_time(),"################第 %s 次迭代####################"%index)
    group_data_dict, group_label_dict, center_data_array,center_data_array1 = iteration_cluster(group_data_dict, group_label_dict, center_data_array, 15,3)
    


 2019-09-17 19:31:00.090447 :  ################第 0 次迭代####################
解散后：
[4.3 3.  1.1 0.1] 50 ---- [4.3 3.  1.1 0.1]
[5.8 2.7 3.9 1.2] 72 ---- [5.8 2.7 3.9 1.2]
[7.9 3.8 6.4 2. ] 28 ---- [7.9 3.8 6.4 2. ]
[[5.97638888 2.78611112 4.51805554 1.49444443]] [[6.99642854 3.09285712 5.9035714  2.14285712]]
[[5.97638888 2.78611112 4.51805554 1.49444443]
 [6.99642854 3.09285712 5.9035714  2.14285712]] [[4.75925929 3.18148149 1.42962962 0.21851852]]
[[5.97638888 2.78611112 4.51805554 1.49444443]
 [6.99642854 3.09285712 5.9035714  2.14285712]
 [4.75925929 3.18148149 1.42962962 0.21851852]] [[5.29565214 3.71739131 1.5        0.27826088]]
拆分后：
[5.80000019 2.70000005 3.9000001  1.20000005] 72 ---- [5.97638888 2.78611112 4.51805554 1.49444443]
[7.9000001  3.79999995 6.4000001  2.        ] 28 ---- [6.99642854 3.09285712 5.9035714  2.14285712]
[4.30000019 3.         1.10000002 0.1       ] 27 ---- [4.75925929 3.18148149 1.42962962 0.21851852]
[5.80000019 4.         1.20000005 0.2       ] 23 ----

In [39]:
for i in group_label_dict:
    for j in group_label_dict[i]:
        iris.ix[j,"predict"]=i
print(np.array(iris["label"]),"\n",np.array(iris["predict"]).astype(int))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2] 
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 1 2 1 2 1 1 2 1 1 1 2
 2 1 2 2 2 2 1 1 2 1 2 1 2 1 1 2 2 2 1 1 1 2 2 2 1 2 2 2 1 1 2 2 1 1 2 2 2
 2 2]
